Demonstrate delay correction in Scan

For this notebook, we use the "Dual 0" chain for mkids software.  I

On the ZCU216 the bitfile mkids_2x2_kidsim_v2 uses these connections:
* DAC output on DAC Tile = 2, DAC Ch = 0, which is the connector 0_230, on JHC3
* ADC input on ADC Tile = 2, ADC Ch = 0, which is the connector 0_226, on JHC7

We use the "Sim 0" chain to simulate the resonance.  It uses these connections:
* DAC output on DAC Tile = 3, DAC Ch = 0, which is the connector 0_231, on JHC3
* ADC input on ADC Tile = 3, ADC Ch = 0, which is the connector 0_227, on JHC7

These are connected through the low-frequency Baluns.

connector 0_230, on JHC3 <--> connector 0_227, on JHC7

connector 0_226, on JHC7 <-->  connector 0_231, on JHC3

In [ ]:
import sys
sys.path.append('../../qick/qick_lib/')
sys.path.append('../soft/')
import Scan
from mkids import *
import matplotlib.pyplot as plt


In [ ]:
# Use this bitfile.  
bitfile = "mkids_2x2_kidsim_v2"

# Set up the firmware to use the kids and simu chains described above
iKids = 0
iSimu = 0
scan = Scan.Scan(bitfile, iKids=iKids, iSimu=iSimu)

In [ ]:
# Here is the function we are demonstrating
scan.sweep_tones?

In [ ]:
# Note that the Scan object include a "nominalDelay" which was determined by running other demo notebooks.  For other combinations of board, firmware, and channels, the Scan class needs to be updated.
print(scan.nominalDelay)

In [ ]:
# Set the mixers so the channel is in the middle of the first nyquist zone.
# Note that the Scan object has the field "fNyquist" which will be set correctly for each board and firmware.
fMixerSimu = 512
fMixerKids = scan.fNyquist/2
print("fNyquist = ",scan.fNyquist)

In [ ]:
# Initialize simulation chain with no resonance
simu = scan.simuChain
simu.analysis.qout(3)
simu.synthesis.qout(3)
simu.alloff()
simu.set_mixer_frequency(fMixerSimu)


In [ ]:
# Initialize the kids chain
kids = scan.kidsChain
# These values for qout work reasonably for loopback.  
kids.analysis.qout(2)
kids.synthesis.qout(2)
kids.set_mixer_frequency(fMixerKids)

In [ ]:
# This function scans over one channel.  Note that its width is gaken from the firmware using the sampling frequency of the pfbs.
def scanOneOutCh(outCh, df=None, N=50, doProgress=False, fMixer=700, doApplyDelay=True, additionalDelay=0):
    scan.set_mixer(fMixer)
    kids = scan.kidsChain
    qFMixer = scan.get_mixer()
    pfbFs = kids.synthesis.dict['chain']['fs']
    fMinimum = qFMixer-pfbFs/2
    fMaximum = qFMixer+pfbFs/2

    fTone = scan.outCh2Freq(outCh)
    if df is None:
        df = kids.synthesis.fc_ch
    fMin = np.maximum(fMinimum, fTone-df/2)
    fMax = np.minimum(fMaximum, fTone+df/2)
    df = fMax-fMin
    fc = (fMin+fMax)/2
    freqs = np.array([fc])
    fis = np.zeros(1)
    gs = 0.9*np.ones(1)
    kids.set_tones(freqs, fis, gs)
    fOffsets = kids.get_sweep_offsets(df, N)
    xs = scan.sweep_tones(freqs, fis, gs, None, df, N, mean=True, doProgress=doProgress, doApplyDelay=doApplyDelay, additionalDelay=additionalDelay)
    return xs, freqs, fOffsets

# Call the function withou applying the delay correction to the phase.
xs, freqs, fOffsets = scanOneOutCh(30, df=0.3, N=50, doProgress=True, fMixer=fMixerKids, doApplyDelay=False)
plt.plot(fOffsets, np.angle(xs), '-o')
plt.xlabel("Frequency = %.1f [MHz]"%freqs[0])
_ = plt.ylabel("Phase [Rad]")

In [ ]:
# Call the function and apply the phase.  This is done in the Scan object function "sweep_tones" which will also do multi-tones and correct for the delays returning calibrated data.

# Residuls from a linear evolution of phase vs. frequency are ~ 1 milli Radians.

xs, freqs, fOffsets = scanOneOutCh(30, df=0.3, N=50, doProgress=True, fMixer=fMixerKids, doApplyDelay=True)
plt.plot(fOffsets, np.angle(xs), '-o')
plt.xlabel("Frequency = %.1f [MHz]"%freqs[0])
_ = plt.ylabel("Phase [Rad]")

In [ ]:
# Call the function and apply the phase specifying an "additionalDelay" to flatten the phase vs. frequency plot a bit more.

xs, freqs, fOffsets = scanOneOutCh(30, df=0.3, N=50, doProgress=True, fMixer=fMixerKids, doApplyDelay=True, additionalDelay=-0.026)
plt.plot(fOffsets, np.angle(xs), '-o')
plt.xlabel("Frequency = %.1f [MHz]"%freqs[0])
_ = plt.ylabel("Phase [Rad]")